In [20]:
import cv2
import mediapipe as mp
import numpy as np
import time
import socketio

In [21]:
sio = socketio.Client()

In [22]:
sio.connect('http://localhost:3000');

In [23]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.1, min_tracking_confidence=0.5,max_num_faces=2, static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)


In [24]:
desired_width = 640  # Set your desired width
desired_height = 480  # Set your desired height

In [25]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    start = time.time()

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (desired_width, desired_height))

    image.flags.writeable = False

    results = face_mesh.process(image)

    image.flags.writeable = True

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    
    atn = 0
    
    if results.multi_face_landmarks:
        for face_id, face_landmarks in enumerate(results.multi_face_landmarks):
            face_3d = []
            face_2d = []
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                            nose_2d = (lm.x * img_w, lm.y * img_h)
                            nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])

            face_2d = np.array(face_2d, dtype=np.float64)

            face_3d = np.array(face_3d, dtype=np.float64)

            focal_length = 1 * img_w
            cam_matrix = np.array([[focal_length, 0, img_h / 2],[0, focal_length, img_w / 2],[0, 0, 1]])

            dist_matrix = np.zeros((4, 1), dtype=np.float64)
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            rmat, jac = cv2.Rodrigues(rot_vec)
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)


            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            

            if y< -10:
                atn += 1
            elif y>10:
                atn += 1
            elif x<-10:
                atn += 1
            elif x>10:
                atn += 1
            else:
                atn += 0
                
                
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))

            cv2.line(image, p1, p2, (255,0,0), 3)
            
            cv2.putText(image, str(atn) , (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),1)

            sio.emit("update_variable", atn)
            time.sleep(0.4)
            

                
            # cv2.putText(image, text + str(face_id), (20, 50 + 100*face_id), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # cv2.putText(image, "x: " + str(np.round(x, 2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),1)
            # cv2.putText(image, "y: " + str(np.round(y, 2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),1)
            # cv2.putText(image, "z: " + str(np.round(z, 2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),1)
            
            mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=drawing_spec,
            connection_drawing_spec= drawing_spec)
        
        end = time.time()
        total_time = end - start

        if total_time > 0:
            fps = 1 / total_time
        else:
            fps = 0

        print("FPS: ", fps)
        cv2.putText(image, f'FPS: {int(fps)}', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        
    cv2.imshow('Head Pose ESTIMATION', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
            

        


FPS:  2.248476732579321
FPS:  2.306979895967918
FPS:  2.249768415090271
FPS:  2.308057696159679
FPS:  2.2479091511783516
FPS:  2.310416114672943
FPS:  2.3565570915203806
FPS:  2.236231034867001
FPS:  2.310860366713681
FPS:  2.3031453171674574
FPS:  2.2218641222498214
FPS:  2.3460779038300834
FPS:  2.2944607067892844
FPS:  2.2532395357395156
FPS:  2.2851900019014626
FPS:  2.286891998802658
FPS:  2.2608792209857342
FPS:  2.3313306686721105
FPS:  2.274870576622165
FPS:  2.3062669940148077
FPS:  2.2733552160885946
FPS:  2.2811445577380343
FPS:  2.2801276865937776
FPS:  2.3078748181456024
FPS:  2.292495545425726
FPS:  2.308381613757706
FPS:  2.300365103018064
FPS:  2.252831680445506
FPS:  2.271027546531303
FPS:  2.274731162968282
FPS:  2.2752901544144972
FPS:  2.264075493941851
FPS:  2.2668409099998597
FPS:  2.263596515371804
FPS:  2.281266146919426
FPS:  1.904568680970633
FPS:  2.2768884345816587
FPS:  2.3035968986628133
FPS:  2.274683050717851
FPS:  2.2744474534433636
FPS:  2.289545983295